### Step 1: Download Data from: http://data.insideairbnb.com/

In [10]:
from urllib.parse import urlparse
import requests
import gzip
from io import BytesIO
import os
import pandas as pd  # Import pandas
import re  # For regular expressions

def clean_state_name(state):
    """Cleans the state name by replacing dashes with spaces and removing unwanted characters."""
    state = state.replace('-', ' ')  # Replace dashes with spaces
    state = re.sub(r'[^a-zA-Z0-9 ]', '', state)  # Remove all characters that are not alphabets, digits, or spaces
    return state

def download_and_process_file(url, base_save_directory):
    # Extracting details from the URL
    parsed_url = urlparse(url)
    path_parts = parsed_url.path.split('/')
    
    # Debugging: Print path_parts
    print("URL path parts:", path_parts)

    # Check if path_parts has enough elements
    if len(path_parts) < 6:
        raise ValueError("URL does not have the expected number of parts")

    try:
        date_part = path_parts[4]
        year, mo = date_part.split('-')[0], date_part.split('-')[1]
        state = clean_state_name(path_parts[2])  # Clean the state name
        city = path_parts[3]
        country = path_parts[1]
        file_type = path_parts[-1].split('.')[0]
    except IndexError as e:
        raise IndexError(f"Error parsing URL: {url}, {e}")

    # Download the file
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to download file: {url}")

    # Extract the file
    with gzip.open(BytesIO(response.content), 'rb') as f_in:
        file_content = f_in.read()

    # Determine the save directory based on file type
    type_directories = {
        'calendar': 'calendar',
        'listings': 'listings',
        'reviews': 'reviews'
    }
    save_directory = os.path.join(base_save_directory, type_directories.get(file_type, 'other'))

    # Ensure the save directory exists
    os.makedirs(save_directory, exist_ok=True)

    # Save the extracted file
    new_filename = f"{year}_{mo}_{city}_{state}_{country}_{file_type}.csv"
    new_filepath = os.path.join(save_directory, new_filename)
    with open(new_filepath, 'wb') as f_out:
        f_out.write(file_content)
    
    return new_filepath

# Base directory to save the processed files
base_save_directory = "./all"

# Read URLs from a CSV file
links_df = pd.read_csv('links.csv')  # Make sure 'links.csv' is in the correct path

# Process the URLs
processed_files = []
for url in links_df.iloc[:, 0]:  # Assume URLs are in the first column
    try:
        processed_file = download_and_process_file(url, base_save_directory)
        processed_files.append(processed_file)
    except Exception as e:
        print(f"Error processing {url}: {e}")
        
# List of processed files
print(processed_files)


URL path parts: ['', 'argentina', 'ciudad-aut%C3%B3noma-de-buenos-aires', 'buenos-aires', '2021-12-25', 'data', 'listings.csv.gz']
URL path parts: ['', 'argentina', 'ciudad-aut%C3%B3noma-de-buenos-aires', 'buenos-aires', '2021-12-25', 'data', 'reviews.csv.gz']
URL path parts: ['', 'australia', 'nsw', 'northern-rivers', '2021-12-12', 'data', 'calendar.csv.gz']
URL path parts: ['', 'australia', 'nsw', 'northern-rivers', '2021-12-12', 'data', 'listings.csv.gz']
URL path parts: ['', 'australia', 'nsw', 'northern-rivers', '2021-12-12', 'data', 'reviews.csv.gz']
URL path parts: ['', 'australia', 'nsw', 'sydney', '2021-12-07', 'data', 'calendar.csv.gz']
URL path parts: ['', 'australia', 'nsw', 'sydney', '2021-12-07', 'data', 'listings.csv.gz']
URL path parts: ['', 'australia', 'nsw', 'sydney', '2021-12-07', 'data', 'reviews.csv.gz']
URL path parts: ['', 'australia', 'sa', 'barossa-valley', '2021-12-24', 'data', 'calendar.csv.gz']
URL path parts: ['', 'australia', 'sa', 'barossa-valley', '2021

In [1]:
import os
import pandas as pd

# Define the path to the 'all/calendar' directory
directory_path = './all/calendar'

# List the files in the directory
files = os.listdir(directory_path)

# Split each file name into components and create a DataFrame
data = []
for file in files:
    parts = file.replace('.csv', '').split('_')
    if len(parts) == 6:  # Ensure the file name has the correct number of parts
        year, mo, city, state, country, _ = parts
        data.append([year, mo, city, state, country, file])  # Include the original file name

# Create DataFrame
columns = ['year', 'mo', 'city', 'state', 'country', 'file_name']
df = pd.DataFrame(data, columns=columns)

# Create a unique identifier for each city-state-country combination
df['city_id'] = pd.factorize(df['city'] + '-' + df['state'] + '-' + df['country'])[0]

# Save the DataFrame as 'data_index.csv'
df.to_csv('./data_index.csv', index=False)


In [3]:
import pandas as pd
import os
from datetime import datetime
import re

# Setup and configurations
input_folder = './all/calendar'
output_folder = './clean/calendar'
index_file = './data_index.csv'
end_date_cutoff = datetime(2023, 12, 31)

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Read city information from index file
def read_city_index():
    print("Reading city index...")
    return pd.read_csv(index_file)

# Function to clean and convert price
def clean_price(price):
    if pd.isnull(price) or price == '':
        return None
    return float(price.strip('$').replace(',', ''))

# Function to convert availability
def convert_availability(available):
    return 1 if available in ['t', 'true'] else 0

# Process individual listings
def process_listing(city, listing_data):
    #print(f"Processing listing {listing_data.iloc[0]['listing_id']} for {city}...")
    listing_data['date'] = pd.to_datetime(listing_data['date'], format='%Y-%m-%d', errors='coerce')
    listing_data = listing_data[listing_data['date'] <= end_date_cutoff]
    listing_data['price'] = listing_data['price'].apply(clean_price)
    listing_data.dropna(subset=['price'], inplace=True)
    listing_data.drop(columns=['adjusted_price'], inplace=True)
    listing_data['available'] = listing_data['available'].apply(convert_availability)
    return listing_data

# Function to extract year and month from filename
def extract_year_month_from_filename(filename):
    match = re.search(r'\d{4}_\d{2}', filename)
    return datetime.strptime(match.group(), '%Y_%m') if match else None

# Process cities by listing_ids
def process_cities_by_listing(city, file_names):
    print(f"Starting processing for city: {city}")
    file_names = sorted(file_names, key=extract_year_month_from_filename, reverse=True)
    all_data = pd.DataFrame()
    for file in file_names:
        print(f"Reading data from file: {file}")
        data = pd.read_csv(f"{input_folder}/{file}")
        all_data = pd.concat([all_data, data])

    grouped_data = all_data.groupby('listing_id')
    processed_data = grouped_data.apply(lambda x: process_listing(city, x)).reset_index(drop=True)
    processed_data.to_csv(f"{output_folder}/Clean_{city}.csv", index=False)
    print(f"Finished processing for {city}.")

# Main script to process a batch of cities
def process_batch_of_cities(cities_batch):
    for city, file_names in cities_batch.items():
        output_path = f"{output_folder}/Clean_{city}.csv"
        if not os.path.exists(output_path):
            process_cities_by_listing(city, file_names)
        else:
            print(f"Skipped processing for {city} as cleaned data already exists.")

# Main script to process all cities
def main():
    print("Data cleaning process started.")
    city_info_df = read_city_index()
    cities_info = city_info_df.groupby('city')['file_name'].apply(list).to_dict()
    cities_batches = [dict(list(cities_info.items())[i:i + 6]) for i in range(0, len(cities_info), 6)]  # Split cities into batches of 6

    for batch in cities_batches:
        process_batch_of_cities(batch)
    print("Data cleaning process completed.")

if __name__ == "__main__":
    main()


Data cleaning process started.
Reading city index...
Skipped processing for albany as cleaned data already exists.
Skipped processing for amsterdam as cleaned data already exists.
Skipped processing for antwerp as cleaned data already exists.
Skipped processing for asheville as cleaned data already exists.
Skipped processing for athens as cleaned data already exists.
Skipped processing for austin as cleaned data already exists.
Skipped processing for bangkok as cleaned data already exists.
Skipped processing for barcelona as cleaned data already exists.
Skipped processing for barossa-valley as cleaned data already exists.
Skipped processing for barwon-south-west-vic as cleaned data already exists.
Skipped processing for beijing as cleaned data already exists.
Skipped processing for belize as cleaned data already exists.
Skipped processing for bergamo as cleaned data already exists.
Skipped processing for berlin as cleaned data already exists.
Skipped processing for bologna as cleaned d

C:\Users\yirsu\AppData\Local\Temp\ipykernel_8492\135682862.py:66: DtypeWarning: Columns (3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  process_cities_by_listing(city, file_names)


Reading data from file: 2023_10_new-york-city_ny_united-states_calendar.csv
Reading data from file: 2023_09_new-york-city_ny_united-states_calendar.csv
Reading data from file: 2023_08_new-york-city_ny_united-states_calendar.csv
Reading data from file: 2023_07_new-york-city_ny_united-states_calendar.csv
Reading data from file: 2023_06_new-york-city_ny_united-states_calendar.csv
Reading data from file: 2023_05_new-york-city_ny_united-states_calendar.csv
Reading data from file: 2023_04_new-york-city_ny_united-states_calendar.csv
Reading data from file: 2023_03_new-york-city_ny_united-states_calendar.csv
Reading data from file: 2023_02_new-york-city_ny_united-states_calendar.csv
Reading data from file: 2023_01_new-york-city_ny_united-states_calendar.csv
Reading data from file: 2022_12_new-york-city_ny_united-states_calendar.csv
Reading data from file: 2022_09_new-york-city_ny_united-states_calendar.csv
Finished processing for new-york-city.
Starting processing for city: newark
Reading data

In [7]:
import os
import pandas as pd

input_folder = './clean/calendar/'

# Function to remove duplicates while keeping the first occurrence
def remove_duplicates(file_path):
    # Read the data
    data = pd.read_csv(file_path)
    
    # Convert 'date' to datetime if it's not already
    data['date'] = pd.to_datetime(data['date'])

    # Remove duplicates while keeping the first occurrence
    cleaned_data = data.drop_duplicates(subset=['listing_id', 'date'], keep='first')

    # Save the cleaned data back to the same file
    cleaned_data.to_csv(file_path, index=False)
    print(f"Processed and saved cleaned data to {file_path}")

# Process all CSV files in the directory
for file_name in os.listdir(input_folder):
    if file_name.endswith('.csv'):
        file_path = input_folder + file_name
        remove_duplicates(file_path)

print("All files have been processed and duplicates removed.")


Processed and saved cleaned data to ./clean/calendar/Clean_albany.csv
Processed and saved cleaned data to ./clean/calendar/Clean_amsterdam.csv
Processed and saved cleaned data to ./clean/calendar/Clean_antwerp.csv
Processed and saved cleaned data to ./clean/calendar/Clean_asheville.csv
Processed and saved cleaned data to ./clean/calendar/Clean_athens.csv
Processed and saved cleaned data to ./clean/calendar/Clean_austin.csv
Processed and saved cleaned data to ./clean/calendar/Clean_bangkok.csv
Processed and saved cleaned data to ./clean/calendar/Clean_barcelona.csv
Processed and saved cleaned data to ./clean/calendar/Clean_barossa-valley.csv
Processed and saved cleaned data to ./clean/calendar/Clean_barwon-south-west-vic.csv
Processed and saved cleaned data to ./clean/calendar/Clean_beijing.csv
Processed and saved cleaned data to ./clean/calendar/Clean_belize.csv
Processed and saved cleaned data to ./clean/calendar/Clean_bergamo.csv
Processed and saved cleaned data to ./clean/calendar/C

In [6]:
import os
import csv
from datetime import date, timedelta
from forex_python.converter import CurrencyRates

# List of country and currency ISO pairs
data = [
    ("belgium", "EUR"), ("australia", "AUD"), ("belize", "BZD"), ("italy", "EUR"),
    ("denmark", "DKK"), ("spain", "EUR"), ("switzerland", "CHF"), ("france", "EUR"),
    ("germany", "EUR"), ("norway", "NOK"), ("latvia", "EUR"), ("singapore", "SGD"),
    ("sweden", "SEK"), ("taiwan", "TWD"), ("greece", "EUR"), ("thailand", "THB"),
    ("china", "CNY"), ("argentina", "ARS"), ("south-africa", "ZAR"), ("portugal", "EUR"),
    ("brazil", "BRL"), ("chile", "CLP"), ("ireland", "EUR"), ("turkey", "TRY"),
    ("the-netherlands", "EUR"), ("canada", "CAD"), ("united-states", "EUR"),
    ("united-kingdom", "EUR"), ("czech-republic", "CZK"), ("austria", "EUR"),
    ("mexico", "MXN"), ("japan", "JPY")
]

# Extract unique ISO codes
unique_iso_codes = sorted(set([iso for _, iso in data]))

# Create a directory for exchange rates if it doesn't exist
folder_path = "./exchange_rate"
os.makedirs(folder_path, exist_ok=True)

# Initialize CurrencyRates
c = CurrencyRates()

# Keep track of ISO codes that have already been processed
processed_isos = set()

# Fetch exchange rate data and save to CSV files
for iso in unique_iso_codes:
    # Skip if this ISO has already been processed
    if iso in processed_isos:
        continue

    file_path = os.path.join(folder_path, f"{iso}.csv")
    success = True  # Flag to indicate if fetching is successful

    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Date", f"Exchange Rate to 1 USD ({iso})"])
        
        # Define the date range
        start_date = date(2021, 12, 1)
        end_date = date(2023, 12, 31)
        current_date = start_date
        
        while current_date <= end_date and success:
            try:
                # Fetch the exchange rate for the current date
                rate = c.get_rate('USD', iso, current_date)
                writer.writerow([current_date, rate])
            except Exception as e:
                print(f"Error fetching data for {iso} on {current_date}: {e}")
                success = False  # Set success to False if any error occurs
            # Move to the next day
            current_date += timedelta(days=1)

    if success:
        print(f"Data for {iso} successfully saved to {file_path}")
    else:
        os.remove(file_path)  # Remove the file if there was an error
        print(f"Data for {iso} was not saved due to errors. File {file_path} has been removed.")

    processed_isos.add(iso)

print("Exchange rate data fetching process completed.")

Error fetching data for ARS on 2021-12-01: Currency Rate USD => ARS not available for Date 2021-12-01
Data for ARS was not saved due to errors. File ./exchange_rate\ARS.csv has been removed.
Data for AUD successfully saved to ./exchange_rate\AUD.csv
Data for BRL successfully saved to ./exchange_rate\BRL.csv
Error fetching data for BZD on 2021-12-01: Currency Rate USD => BZD not available for Date 2021-12-01
Data for BZD was not saved due to errors. File ./exchange_rate\BZD.csv has been removed.
Data for CAD successfully saved to ./exchange_rate\CAD.csv
Data for CHF successfully saved to ./exchange_rate\CHF.csv
Error fetching data for CLP on 2021-12-01: Currency Rate USD => CLP not available for Date 2021-12-01
Data for CLP was not saved due to errors. File ./exchange_rate\CLP.csv has been removed.
Data for CNY successfully saved to ./exchange_rate\CNY.csv
Data for CZK successfully saved to ./exchange_rate\CZK.csv
Data for DKK successfully saved to ./exchange_rate\DKK.csv
Error fetching

In [17]:
import os
import pandas as pd
from pathlib import Path
from datetime import datetime

# Function to load and process each file
def process_file(file_path, rates, city_currency_map):
    # Load data
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values(by=['listing_id', 'date'], inplace=True)
    
    # Task 1: Count price changes
    df['price_changed'] = df.groupby('listing_id')['price'].diff().ne(0)
    df['price_change_count'] = df.groupby('listing_id')['price_changed'].cumsum()

    # Task 2: Count availability changes from 0 to 1
    df['availability_changed'] = df['available'].diff().eq(1) & df['listing_id'].eq(df['listing_id'].shift())
    df['change_in_availability'] = df.groupby('listing_id')['availability_changed'].cumsum()

    # Task 3: Calculate monthly and quarterly availability rates
    df['month'] = df['date'].dt.month
    df['quarter'] = df['date'].dt.quarter
    monthly_availability = df.groupby(['listing_id', 'month'])['available'].sum().reset_index(name='monthly_availability_count')
    quarterly_availability = df.groupby(['listing_id', 'quarter'])['available'].sum().reset_index(name='quarterly_availability_count')
    df = df.merge(monthly_availability, on=['listing_id', 'month'], how='left')
    df = df.merge(quarterly_availability, on=['listing_id', 'quarter'], how='left')

    # Task 4: Calculate monthly weekend availability
    df['day_of_week'] = df['date'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)  # Convert to 0/1 dummy variable
    df['weekend_available'] = df['available'] * df['is_weekend']

    # Ensure 'month' column reflects the actual month-year combination
    df['year_month'] = df['date'].dt.to_period('M')

    # Sum weekend availability for each month
    df['monthly_weekend_availability'] = df.groupby(['listing_id', 'year_month'])['weekend_available'].transform('sum')
    # Once used, you can drop the 'year_month' column as it's no longer needed
    df.drop('year_month', axis=1, inplace=True)

    # Task 5: Convert price to USD and create 'price_usd'
    city_name = file_path.split('_')[1].split('.')[0].lower()
    currency = city_currency_map.get(city_name)
    if currency and currency != 'USD':
        exchange_rate_file = f'./exchange_rate/{currency}.csv'
        if os.path.exists(exchange_rate_file):
            exchange_rates = load_exchange_rates(exchange_rate_file)
            df = apply_exchange_rate(df, exchange_rates, f'Exchange Rate to 1 USD ({currency})')
        else:
            print(f"No exchange rate data found for currency: {currency}")
    else:
        df['price_usd'] = df['price']  # For USD, copy the price

    # Save the processed data
    new_file_path = file_path.replace('/calendar/', '/calendar2/')
    df.to_csv(new_file_path, index=False)
    print(f"Processed and saved to {new_file_path}")

# Load exchange rates
def load_exchange_rates(file_path):
    rates = pd.read_csv(file_path, parse_dates=['Date'])
    rates['Date'] = rates['Date'].dt.date
    rates.set_index('Date', inplace=True)
    return rates

# Apply exchange rate to convert price to USD
def apply_exchange_rate(df, rates, currency_column):
    df['date'] = df['date'].dt.date
    df = df.merge(rates, left_on='date', right_index=True, how='left')
    df['price_usd'] = (df['price'] / df[currency_column]).round(3)
    df.drop(columns=[currency_column], inplace=True)
    return df

# Load the data index to map cities to currencies
data_index = pd.read_csv('./data_index.csv')
city_currency_map = dict(zip(data_index['city'].str.lower(), data_index['currency_iso']))

# Create the output directory if it doesn't exist
Path('./clean/calendar2/').mkdir(parents=True, exist_ok=True)

# Process each file in the directory
for file_name in os.listdir('./clean/calendar/'):
    if file_name.endswith('.csv'):
        file_path = f'./clean/calendar/{file_name}'
        process_file(file_path, None, city_currency_map)  # Pass None for rates as it's loaded in process_file


Processed and saved to ./clean/calendar2/Clean_albany.csv
Processed and saved to ./clean/calendar2/Clean_amsterdam.csv
Processed and saved to ./clean/calendar2/Clean_antwerp.csv
Processed and saved to ./clean/calendar2/Clean_asheville.csv
Processed and saved to ./clean/calendar2/Clean_athens.csv
Processed and saved to ./clean/calendar2/Clean_austin.csv
Processed and saved to ./clean/calendar2/Clean_bangkok.csv
Processed and saved to ./clean/calendar2/Clean_barcelona.csv
Processed and saved to ./clean/calendar2/Clean_barossa-valley.csv
Processed and saved to ./clean/calendar2/Clean_barwon-south-west-vic.csv
Processed and saved to ./clean/calendar2/Clean_beijing.csv
Processed and saved to ./clean/calendar2/Clean_belize.csv
Processed and saved to ./clean/calendar2/Clean_bergamo.csv
Processed and saved to ./clean/calendar2/Clean_berlin.csv
Processed and saved to ./clean/calendar2/Clean_bologna.csv
Processed and saved to ./clean/calendar2/Clean_bordeaux.csv
Processed and saved to ./clean/ca

In [56]:
import pandas as pd
import os
import glob
import re

# Global dictionaries to keep track of category indices
global_category_indices = {
    'host_verifications': {},
    'host_response_time': {},
    'property_type': {},
    'room_type': {},
    'neighbourhood_cleansed': {}
}

# Function to convert percentages to decimals
def percentage_to_decimal(s):
    if isinstance(s, str) and s.endswith('%'):
        return float(s.strip('%')) / 100
    return s

# Function to convert categories to numeric indices
def category_to_index(cat, global_dict):
    if cat not in global_dict:
        global_dict[cat] = len(global_dict) + 1  # Start indexing from 1
    return global_dict[cat]

# Function to save category indices to a file
def save_category_indices(global_dict, path):
    pd.Series(global_dict).to_csv(path)

# Function to process and merge files
def process_and_merge_files(calendar_file, listings_file, output_file, 
                            last_scraped_start_date=None, last_scraped_end_date=None, 
                            mode='w', header=True, index_folder='./index_files/'):
    print(f"Processing: {calendar_file} and {listings_file}")
    
    # Read the calendar and listings data
    calendar_df = pd.read_csv(calendar_file)
    listings_df = pd.read_csv(listings_file)

    # Convert 'date' in calendar_df to datetime and ensure IDs are of the same type
    calendar_df['date'] = pd.to_datetime(calendar_df['date'], errors='coerce')
    calendar_df['listing_id'] = calendar_df['listing_id'].astype(str)
    listings_df['id'] = listings_df['id'].astype(str)
    
    # Ensure last_scraped is a datetime in listings_df for comparison
    listings_df['last_scraped'] = pd.to_datetime(listings_df['last_scraped'], errors='coerce')

    # Function to count words safely, considering non-string values
    def safe_word_count(x):
        if isinstance(x, str):
            return len(x.split())
        else:
            return 0  # Or return np.nan if you prefer
        
    listings_df['name_c'] = listings_df['name'].apply(safe_word_count)
    listings_df['description_c'] = listings_df['description'].apply(safe_word_count)
    listings_df['neighborhood_overview_c'] = listings_df['neighborhood_overview'].apply(safe_word_count)
    listings_df['host_about_c'] = listings_df['host_about'].apply(safe_word_count)
    listings_df['host_listings_count_airbnb'] = listings_df['host_listings_count']

    # Convert 't' and 'f' to 1 and 0
    for col in listings_df.columns:
        unique_values = listings_df[col].dropna().unique()
        if set(unique_values) == {'t', 'f'}:
            listings_df[col] = listings_df[col].map({'t': 1, 'f': 0})

    # Convert 'bathrooms_text' to numerical
    listings_df['bathrooms'] = listings_df['bathrooms_text'].str.extract('(\d+)').astype(float)

    # Count the number of amenities and add as a new column
    listings_df['amenities_count'] = listings_df['amenities'].str.count(',') + 1

    # Drop the original 'bathrooms_text' and 'amenities' columns
    listings_df.drop(columns=['bathrooms_text', 'amenities', 'name', 'description','neighborhood_overview', 'host_about'], inplace=True)

    # Convert specific columns to binary (0 or 1)
    listings_df['license'] = listings_df['license'].notna().astype(int)

    # Fill missing values in 'bathrooms' with 0
    listings_df['bathrooms'].fillna(0, inplace=True)

    # Convert 'host_response_rate' from percentage to decimal
    listings_df['host_response_rate'] = listings_df['host_response_rate'].apply(percentage_to_decimal)

    # Convert categorical variables to numeric indices and update global indices
    for col in ['host_verifications', 'host_response_time', 'property_type', 'room_type']:
        listings_df[col] = listings_df[col].apply(lambda x: category_to_index(x, global_category_indices[col]))

    # Convert 'neighbourhood_cleansed' to numeric index, treated separately for each city
    listings_df['neighbourhood_cleansed'] = listings_df['neighbourhood_cleansed'].apply(lambda x: category_to_index(x, global_category_indices['neighbourhood_cleansed']))

    # Keep only necessary columns from calendar and listings data
    calendar_df = calendar_df[['listing_id', 'date', 'available', 'price', 'minimum_nights', 'maximum_nights',	'price_change_count', 
                               'change_in_availability', 'month', 'quarter', 'monthly_availability_count', 'quarterly_availability_count',
                               'is_weekend', 'monthly_weekend_availability', 'price_usd']]
    listings_columns_to_keep = [
            'id',
            'last_scraped',
            'name_c', 
            'description_c', 
            'neighborhood_overview_c', 
            'host_listings_count_airbnb',             
            'host_id', 
            'host_since', 
            'host_about_c', 
            'host_response_time', 
            'host_response_rate', 
            'host_acceptance_rate', 
            'host_is_superhost', 
            'host_verifications', 
            'host_has_profile_pic', 
            'host_identity_verified', 
            'neighbourhood_cleansed', 
            'property_type', 
            'room_type', 
            'accommodates', 
            'bathrooms', 
            'bedrooms', 
            'beds', 
            'amenities_count', 
            'minimum_minimum_nights', 
            'maximum_minimum_nights', 
            'minimum_maximum_nights', 
            'maximum_maximum_nights', 
            'minimum_nights_avg_ntm', 
            'maximum_nights_avg_ntm', 
            'availability_30', 
            'availability_60', 
            'availability_90', 
            'availability_365', 
            'number_of_reviews', 
            'number_of_reviews_ltm', 
            'number_of_reviews_l30d', 
            'first_review', 
            'last_review', 
            'review_scores_rating', 
            'review_scores_accuracy', 
            'review_scores_cleanliness', 
            'review_scores_checkin', 
            'review_scores_communication', 
            'review_scores_location', 
            'review_scores_value', 
            'license', 
            'instant_bookable', 
            'calculated_host_listings_count', 
            'calculated_host_listings_count_entire_homes', 
            'calculated_host_listings_count_private_rooms', 
            'calculated_host_listings_count_shared_rooms', 
            'reviews_per_month', 
        ]

    listings_df = listings_df[listings_columns_to_keep]

    # Merge logic based on 'listing_id' from calendar_df and 'id' from listings_df
    merged_data = pd.merge(calendar_df, listings_df, left_on='listing_id', right_on='id')
    
    # Convert last_scraped_start_date and last_scraped_end_date to datetime for comparison
    if last_scraped_start_date:
        last_scraped_start_date = pd.to_datetime(last_scraped_start_date, errors='coerce')
    if last_scraped_end_date:
        last_scraped_end_date = pd.to_datetime(last_scraped_end_date, errors='coerce')

    # Filter data based on 'last_scraped' and the provided date ranges
    if last_scraped_start_date is not None and last_scraped_end_date is not None:
        merged_data = merged_data[
            (merged_data['date'] >= last_scraped_start_date) & 
            (merged_data['date'] < last_scraped_end_date)
        ]
    elif last_scraped_start_date is not None:
        # This is for the most recent file where the end date is 2023-12-31
        merged_data = merged_data[
            (merged_data['date'] >= last_scraped_start_date) & 
            (merged_data['date'] <= pd.to_datetime('2023-12-31'))
        ]

    # Save the merged data to the specified output file
    merged_data.to_csv(output_file, mode=mode, header=header, index=False)
    if mode == 'a':
        print(f"Appended data to {output_file}")
    else:
        print(f"Saved new data to {output_file}")

    # Save category indices to files
    if not os.path.exists(index_folder):
        os.makedirs(index_folder)
    for cat, idx_dict in global_category_indices.items():
        save_category_indices(idx_dict, os.path.join(index_folder, f'{cat}_index.csv'))

# Function to iterate over files and merge them for a specific city
def merge_city_data(city_name, calendar_folder, listings_folder, output_folder):
    calendar_files = sorted(glob.glob(f'{calendar_folder}/Clean_{city_name}.csv'))
    listings_files = sorted(glob.glob(os.path.join(listings_folder, f'*{city_name}*_listings.csv')), reverse=True)

    last_scraped_end_date = pd.to_datetime('2023-12-31')  # Initial end date for the most recent file
    last_scraped_start_date = None  # Initialize this variable for the first iteration

    for listings_file in listings_files:
        output_file = os.path.join(output_folder, 'merged_' + city_name + '.csv')
        
        calendar_file = calendar_files[0] if calendar_files else None
        
        if not calendar_file:
            print(f"No matching calendar file found for {listings_file}. Skipping.")
            continue

        # Update the last_scraped_start_date for the current file
        current_last_scraped = pd.read_csv(listings_file)['last_scraped'].min()
        current_last_scraped = pd.to_datetime(current_last_scraped, errors='coerce')

        # Determine whether to write a new file or append to the existing one
        if os.path.exists(output_file):
            mode = 'a'  # Append if already exists
            header = False  # Don't write the header again
        else:
            mode = 'w'  # Create a new file if not
            header = True

        # Process the files using the adjusted date ranges
        process_and_merge_files(calendar_file, listings_file, output_file, 
                                last_scraped_start_date, last_scraped_end_date, mode, header)

        # Update the last_scraped_end_date for the next iteration to be the start date of the current one
        last_scraped_start_date = current_last_scraped
        last_scraped_end_date = current_last_scraped - pd.Timedelta(days=1)  # Set end date to the day before the current last_scraped

# Function to extract city names from the listings files
def get_city_names_from_files(listings_folder):
    city_names = set()
    files = glob.glob(f'{listings_folder}/*_listings.csv')
    for file in files:
        match = re.search(r'\d{4}_\d{2}_(.*?)_.*_.*_listings\.csv', os.path.basename(file))
        if match:
            city_names.add(match.group(1))
    return city_names

# Main script to iterate over all cities and merge data
def merge_all_cities():
    calendar_folder = './clean/calendar2'
    listings_folder = './all/listings'
    output_folder = './merge/'

    # Extract city names from the listings files
    city_names = get_city_names_from_files(listings_folder)

    # Process each city
    for city_name in city_names:
        print(f"Starting merge process for {city_name}")
        merge_city_data(city_name, calendar_folder, listings_folder, output_folder)

# Start the process for all cities
merge_all_cities()

Starting merge process for portland
Processing: ./clean/calendar2/Clean_portland.csv and ./all/listings\2023_09_portland_or_united-states_listings.csv
Saved new data to ./merge/merged_portland.csv
Processing: ./clean/calendar2/Clean_portland.csv and ./all/listings\2023_06_portland_or_united-states_listings.csv
Appended data to ./merge/merged_portland.csv
Processing: ./clean/calendar2/Clean_portland.csv and ./all/listings\2023_03_portland_or_united-states_listings.csv
Appended data to ./merge/merged_portland.csv
Processing: ./clean/calendar2/Clean_portland.csv and ./all/listings\2022_12_portland_or_united-states_listings.csv
Appended data to ./merge/merged_portland.csv
Processing: ./clean/calendar2/Clean_portland.csv and ./all/listings\2022_09_portland_or_united-states_listings.csv
Appended data to ./merge/merged_portland.csv
Starting merge process for madrid
Processing: ./clean/calendar2/Clean_madrid.csv and ./all/listings\2023_09_madrid_comunidad de madrid_spain_listings.csv
Saved new

C:\Users\yirsu\AppData\Local\Temp\ipykernel_29184\2867873049.py:255: DtypeWarning: Columns (68) have mixed types.Specify dtype option on import or set low_memory=False.
  merge_city_data(city_name, calendar_folder, listings_folder, output_folder)


Processing: ./clean/calendar2/Clean_new-york-city.csv and ./all/listings\2023_05_new-york-city_ny_united-states_listings.csv


C:\Users\yirsu\AppData\Local\Temp\ipykernel_29184\2867873049.py:220: DtypeWarning: Columns (68) have mixed types.Specify dtype option on import or set low_memory=False.
  process_and_merge_files(calendar_file, listings_file, output_file,


Appended data to ./merge/merged_new-york-city.csv
Processing: ./clean/calendar2/Clean_new-york-city.csv and ./all/listings\2023_04_new-york-city_ny_united-states_listings.csv
Appended data to ./merge/merged_new-york-city.csv
Processing: ./clean/calendar2/Clean_new-york-city.csv and ./all/listings\2023_03_new-york-city_ny_united-states_listings.csv
Appended data to ./merge/merged_new-york-city.csv
Processing: ./clean/calendar2/Clean_new-york-city.csv and ./all/listings\2023_02_new-york-city_ny_united-states_listings.csv
Appended data to ./merge/merged_new-york-city.csv
Processing: ./clean/calendar2/Clean_new-york-city.csv and ./all/listings\2023_01_new-york-city_ny_united-states_listings.csv
Appended data to ./merge/merged_new-york-city.csv
Processing: ./clean/calendar2/Clean_new-york-city.csv and ./all/listings\2022_12_new-york-city_ny_united-states_listings.csv
Appended data to ./merge/merged_new-york-city.csv
Processing: ./clean/calendar2/Clean_new-york-city.csv and ./all/listings\20

...

Appended data to ./merge/merged_cape-town.csv
Starting merge process for malaga
Processing: ./clean/calendar2/Clean_malaga.csv and ./all/listings\2023_09_malaga_andalucC3ADa_spain_listings.csv
Saved new data to ./merge/merged_malaga.csv
Processing: ./clean/calendar2/Clean_malaga.csv and ./all/listings\2023_06_malaga_andalucC3ADa_spain_listings.csv
Appended data to ./merge/merged_malaga.csv
Processing: ./clean/calendar2/Clean_malaga.csv and ./all/listings\2023_03_malaga_andalucC3ADa_spain_listings.csv
Appended data to ./merge/merged_malaga.csv
Processing: ./clean/calendar2/Clean_malaga.csv and ./all/listings\2022_12_malaga_andalucC3ADa_spain_listings.csv
Appended data to ./merge/merged_malaga.csv
Processing: ./clean/calendar2/Clean_malaga.csv and ./all/listings\2022_09_malaga_andalucC3ADa_spain_listings.csv
Appended data to ./merge/merged_malaga.csv
Processing: ./clean/calendar2/Clean_malaga.csv and ./all/listings\2022_06_malaga_andalucC3ADa_spain_listings.csv
Appended data to ./merge/m

C:\Users\yirsu\AppData\Local\Temp\ipykernel_29184\2867873049.py:255: DtypeWarning: Columns (7,21,27,59,60) have mixed types.Specify dtype option on import or set low_memory=False.
  merge_city_data(city_name, calendar_folder, listings_folder, output_folder)


Processing: ./clean/calendar2/Clean_trentino.csv and ./all/listings\2023_09_trentino_trentino alto adige sC3BCdtirol_italy_listings.csv


C:\Users\yirsu\AppData\Local\Temp\ipykernel_29184\2867873049.py:220: DtypeWarning: Columns (7,21,27,59,60) have mixed types.Specify dtype option on import or set low_memory=False.
  process_and_merge_files(calendar_file, listings_file, output_file,


Saved new data to ./merge/merged_trentino.csv
Processing: ./clean/calendar2/Clean_trentino.csv and ./all/listings\2023_06_trentino_trentino alto adige sC3BCdtirol_italy_listings.csv
Appended data to ./merge/merged_trentino.csv
Processing: ./clean/calendar2/Clean_trentino.csv and ./all/listings\2023_03_trentino_trentino alto adige sC3BCdtirol_italy_listings.csv
Appended data to ./merge/merged_trentino.csv
Processing: ./clean/calendar2/Clean_trentino.csv and ./all/listings\2022_12_trentino_trentino alto adige sC3BCdtirol_italy_listings.csv
Appended data to ./merge/merged_trentino.csv
Processing: ./clean/calendar2/Clean_trentino.csv and ./all/listings\2022_09_trentino_trentino alto adige sC3BCdtirol_italy_listings.csv
Appended data to ./merge/merged_trentino.csv
Processing: ./clean/calendar2/Clean_trentino.csv and ./all/listings\2022_06_trentino_trentino alto adige sC3BCdtirol_italy_listings.csv
Appended data to ./merge/merged_trentino.csv
Processing: ./clean/calendar2/Clean_trentino.csv 

In [13]:
import pandas as pd
import numpy as np
import os
import re

# Define the directory paths
input_dir = './merge/'
output_dir = './2022_data/'

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define the columns to keep
columns_to_keep = ['listing_id', 'date', 'available', 'price', 'price_change_count', 'change_in_availability', 'month', 
                'quarter', 'monthly_availability_count', 'quarterly_availability_count', 'monthly_weekend_availability',
                'price_usd', 'name_c', 'description_c', 'neighborhood_overview_c', 'is_weekend', 'minimum_nights', 'maximum_nights', 
                'host_listings_count_airbnb', 'host_id', 'host_since', 'host_about_c', 'host_response_time', 'host_response_rate', 
                'host_acceptance_rate', 'host_is_superhost', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 
                'neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities_count', 
                'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 
                'maximum_nights_avg_ntm', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'number_of_reviews', 
                'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 
                'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 
                'license', 'instant_bookable', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 
                'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'reviews_per_month',
        ]

# Function to convert percentage strings to decimals
def percentage_to_decimal(s):
    if isinstance(s, str) and s.endswith('%'):
        return float(s.strip('%')) / 100
    return s

# Function to process each file
def process_file(file_path):
    city_name = re.search(r'merged_(.+)\.csv', file_path).group(1)
    data = pd.read_csv(file_path, parse_dates=['date'], low_memory=False)
    
    # Filter rows by date range
    mask = (data['date'] >= '2021-12-01') & (data['date'] <= '2022-12-31')
    data = data.loc[mask]
    
    # Keep only the specified columns
    data = data[columns_to_keep]
    
    # Replace missing values with zeros instead of dropping rows
    data[columns_to_keep] = data[columns_to_keep].fillna(0)
    
    # Convert 'host_acceptance_rate' from percentage to decimal
    data['host_acceptance_rate'] = data['host_acceptance_rate'].apply(percentage_to_decimal)

    # Check and convert 'host_has_profile_pic' from 't'/'f' to 1/0 if necessary
    if 'host_has_profile_pic' in data.columns:
        unique_values = data['host_has_profile_pic'].dropna().unique()
        if all(val in ['t', 'f'] for val in unique_values):
            data['host_has_profile_pic'] = data['host_has_profile_pic'].map({'t': 1, 'f': 0})
        elif any(val not in [1, 0] for val in unique_values):
            print(f"Warning: Unexpected values in 'host_has_profile_pic' column in {file_path}")

    # Data Cleaning and Feature Engineering
    data['host_since_days'] = (pd.to_datetime('today') - pd.to_datetime(data['host_since'])).dt.days
    data['first_review_days'] = (pd.to_datetime('today') - pd.to_datetime(data['first_review'])).dt.days
    data['last_review_days'] = (pd.to_datetime('today') - pd.to_datetime(data['last_review'])).dt.days
    
    # Save the filtered data
    output_file_path = os.path.join(output_dir, f'final_2022_{city_name}.csv')
    data.to_csv(output_file_path, index=False)
    print(f"Processed and saved: {output_file_path}")

# Process each file in the input directory
for file_name in os.listdir(input_dir):
    if file_name.startswith('merged_') and file_name.endswith('.csv'):
        process_file(os.path.join(input_dir, file_name))

print("Processing complete!")

Processed and saved: ./2022_data/final_2022_albany.csv
Processed and saved: ./2022_data/final_2022_amsterdam.csv
Processed and saved: ./2022_data/final_2022_antwerp.csv
Processed and saved: ./2022_data/final_2022_asheville.csv
Processed and saved: ./2022_data/final_2022_athens.csv
Processed and saved: ./2022_data/final_2022_austin.csv
Processed and saved: ./2022_data/final_2022_bangkok.csv
Processed and saved: ./2022_data/final_2022_barcelona.csv
Processed and saved: ./2022_data/final_2022_barossa-valley.csv
Processed and saved: ./2022_data/final_2022_barwon-south-west-vic.csv
Processed and saved: ./2022_data/final_2022_belize.csv
Processed and saved: ./2022_data/final_2022_bergamo.csv
Processed and saved: ./2022_data/final_2022_berlin.csv
Processed and saved: ./2022_data/final_2022_bologna.csv
Processed and saved: ./2022_data/final_2022_bordeaux.csv
Processed and saved: ./2022_data/final_2022_boston.csv
Processed and saved: ./2022_data/final_2022_bozeman.csv
Processed and saved: ./202

In [31]:
import pandas as pd
import numpy as np
import os
import re

# Define the directory paths
input_dir = './merge/'
output_dir = './2023_data/'

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define the columns to keep
columns_to_keep = ['listing_id', 'date', 'available', 'price', 'price_change_count', 'change_in_availability', 'month', 
                'quarter', 'monthly_availability_count', 'quarterly_availability_count', 'monthly_weekend_availability',
                'price_usd', 'name_c', 'description_c', 'neighborhood_overview_c', 'is_weekend', 'minimum_nights', 'maximum_nights', 
                'host_listings_count_airbnb', 'host_id', 'host_since', 'host_about_c', 'host_response_time', 'host_response_rate', 
                'host_acceptance_rate', 'host_is_superhost', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 
                'neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities_count', 
                'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 
                'maximum_nights_avg_ntm', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'number_of_reviews', 
                'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 
                'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 
                'license', 'instant_bookable', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 
                'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'reviews_per_month',
        ]

# Function to convert percentage strings to decimals
def percentage_to_decimal(s):
    if isinstance(s, str) and s.endswith('%'):
        return float(s.strip('%')) / 100
    return s

# Function to process each file
def process_file(file_path):
    city_name = re.search(r'merged_(.+)\.csv', file_path).group(1)
    data = pd.read_csv(file_path, parse_dates=['date'], low_memory=False)
    
    # Filter rows by date range
    mask = (data['date'] >= '2023-01-01') & (data['date'] <= '2023-09-01')
    data = data.loc[mask]
    
    # Keep only the specified columns
    data = data[columns_to_keep]
    
    # Replace missing values with zeros instead of dropping rows
    data[columns_to_keep] = data[columns_to_keep].fillna(0)
    
    # Convert 'host_acceptance_rate' from percentage to decimal
    data['host_acceptance_rate'] = data['host_acceptance_rate'].apply(percentage_to_decimal)

    # Check and convert 'host_has_profile_pic' from 't'/'f' to 1/0 if necessary
    if 'host_has_profile_pic' in data.columns:
        unique_values = data['host_has_profile_pic'].dropna().unique()
        if all(val in ['t', 'f'] for val in unique_values):
            data['host_has_profile_pic'] = data['host_has_profile_pic'].map({'t': 1, 'f': 0})
        elif any(val not in [1, 0] for val in unique_values):
            print(f"Warning: Unexpected values in 'host_has_profile_pic' column in {file_path}")

    # Data Cleaning and Feature Engineering
    data['host_since_days'] = (pd.to_datetime('today') - pd.to_datetime(data['host_since'], errors='coerce')).dt.days
    data['first_review_days'] = (pd.to_datetime('today') - pd.to_datetime(data['first_review'], errors='coerce')).dt.days
    data['last_review_days'] = (pd.to_datetime('today') - pd.to_datetime(data['last_review'], errors='coerce')).dt.days

    # Replace NaN values with 0
    data[['host_since_days', 'first_review_days', 'last_review_days']] = data[['host_since_days', 'first_review_days', 'last_review_days']].fillna(0)

    
    # Save the filtered data
    output_file_path = os.path.join(output_dir, f'final_2023_{city_name}.csv')
    data.to_csv(output_file_path, index=False)
    print(f"Processed and saved: {output_file_path}")

# Process each file in the input directory
for file_name in os.listdir(input_dir):
    if file_name.startswith('merged_') and file_name.endswith('.csv'):
        process_file(os.path.join(input_dir, file_name))

print("Processing complete!")

Processed and saved: ./2023_data/final_2023_albany.csv
Processed and saved: ./2023_data/final_2023_amsterdam.csv
Processed and saved: ./2023_data/final_2023_antwerp.csv
Processed and saved: ./2023_data/final_2023_asheville.csv
Processed and saved: ./2023_data/final_2023_athens.csv
Processed and saved: ./2023_data/final_2023_austin.csv
Processed and saved: ./2023_data/final_2023_bangkok.csv
Processed and saved: ./2023_data/final_2023_barcelona.csv
Processed and saved: ./2023_data/final_2023_barossa-valley.csv
Processed and saved: ./2023_data/final_2023_barwon-south-west-vic.csv
Processed and saved: ./2023_data/final_2023_belize.csv
Processed and saved: ./2023_data/final_2023_bergamo.csv
Processed and saved: ./2023_data/final_2023_berlin.csv
Processed and saved: ./2023_data/final_2023_bologna.csv
Processed and saved: ./2023_data/final_2023_bordeaux.csv
Processed and saved: ./2023_data/final_2023_boston.csv
Processed and saved: ./2023_data/final_2023_bozeman.csv
Processed and saved: ./202